In [1]:
import geopandas as gpd
import pandas as pd
import folium
from folium.plugins import HeatMap

In [2]:
gdf = gpd.read_file('russia.geojson')

In [3]:
df = pd.read_csv('data/data_agglomerative.csv', low_memory=False)

Заменим номер кластеров так, чтобы 1 был самый плохой, а 6 самый лучший

In [4]:
new_order = {
    3: 6, 
    1: 5, 
    5: 4, 
    4: 3, 
    0: 2, 
    2: 1,
}
df['cluster'] = df['cluster'].map(new_order)

Подсчитаем количество объектов в каждом регионе

In [5]:
df_cluster = df.copy()
objects_by_region = df_cluster['Region'].value_counts().reset_index()
objects_by_region.columns = ['Region', 'object_count']
objects_by_region

,Region,object_count
0,Краснодарский край,25
1,Республика Татарстан (Татарстан),24
2,г. Санкт-Петербург,21
3,Свердловская область,21
4,Воронежская область,18
5,Приморский край,18
6,Новосибирская область,17
7,Кемеровская область,16
8,Самарская область,15
9,Нижегородская область,14


In [6]:
# Присоединим данные к GeoDataFrame
gdf_regions = gdf.merge(objects_by_region, how="left", left_on='region', right_on='Region')

In [7]:
# Если в регионах нет объектов, заполним нулями
gdf_regions['object_count'] = gdf_regions['object_count'].fillna(0)

In [8]:
gdf_regions

,region,geometry,Region,object_count
0,Алтайский край,"POLYGON ((82.80364 50.94067, 82.78856 50.94470...",NaN,0.0
1,Амурская область,"POLYGON ((131.38809 49.25325, 131.28198 49.250...",Амурская область,3.0
2,Архангельская область,"MULTIPOLYGON (((36.77240 63.91510, 36.77083 63...",NaN,0.0
3,Астраханская область,"POLYGON ((47.59436 45.44006, 47.58152 45.44518...",NaN,0.0
4,Белгородская область,"POLYGON ((38.93024 49.82358, 38.92225 49.83007...",Белгородская область,6.0
...,...,...,...,...
80,Ямало-Ненецкий автономный округ,"MULTIPOLYGON (((65.99583 69.06250, 65.99375 69...",Ямало-Ненецкий автономный округ,1.0
81,Ярославская область,"POLYGON ((38.32256 56.75940, 38.27213 56.80248...",Ярославская область,9.0
82,г. Москва,"MULTIPOLYGON (((37.24789 55.58891, 37.24775 55...",NaN,0.0
83,г. Санкт-Петербург,"MULTIPOLYGON (((29.71917 59.97542, 29.71917 59...",г. Санкт-Петербург,21.0


In [9]:

latitude = 55
longitude = 37
m = folium.Map(location=[latitude, longitude], zoom_start=10) 

In [10]:

heat_data = [(row.geometry.centroid.y, row.geometry.centroid.x, row.object_count)
             for idx, row in gdf_regions.iterrows() if row.object_count > 0]

HeatMap(heat_data).add_to(m)

m.save('heatmaps/heatmap_all.html')

Сделаем файлики с тепловыми картами для каждого кластера

In [11]:
latitude = 55
longitude = 37
for cluster in range(1, df_cluster['cluster'].max() + 1):
    m = folium.Map(location=[latitude, longitude], zoom_start=10)
    objects_by_region = df_cluster[df_cluster['cluster']==cluster]['Region'].value_counts().reset_index()
    objects_by_region.columns = ['Region', 'object_count']
    gdf_regions = gdf.merge(objects_by_region, how="left", left_on='region', right_on='Region')
    gdf_regions['object_count'] = gdf_regions['object_count'].fillna(0)
    
    heat_data = [(row.geometry.centroid.y, row.geometry.centroid.x, row.object_count)
                 for idx, row in gdf_regions.iterrows() if row.object_count > 0]
    
    HeatMap(heat_data).add_to(m)
    
    m.save(f'heatmaps/heatmap_{cluster}.html')